In [3]:
import pandas as pd
from os import getcwd

import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans


import matplotlib.pyplot as plt 

getcwd()

'c:\\Users\\vicky\\Downloads\\project_04-main\\project_04-main\\depreciated_ipynb'

# Clean the data

-----

In [ ]:
df = pd.read_csv('cleaned_sedan_data.csv')

df.head()

In [ ]:
df.info()

In [ ]:
# Find null values
for column in df.columns:
    print(f"Column - {column} has - {df[column].isnull().sum()} - null values")

In [ ]:
df.count()

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=['vf_EngineCylinders','vf_EngineHP','vf_BodyCabType', 'vin','msrp','vf_Doors','vf_FuelTypePrimary','vf_TransmissionStyle','color'])

In [ ]:
# Find null values
for column in df.columns:
    print(f"Column - {column} has {df[column].isnull().sum()} null values")

In [ ]:
df = df.dropna(how='any')

In [ ]:
old_list = df.columns.to_list()

new_list =[]

for i, column in enumerate(old_list):
    words = column.split('_')
    keep = words[-1]
    keep = keep.lower()
    new_list.append(keep)

In [ ]:
df = df.rename(columns={
 'vf_BodyClass':'bodyclass',
 'vf_Make':'make',
 'vf_Model':'model',
 'vf_ModelID':'modelid',
 'vf_ModelYear':'modelyear',
 'mileage':'mileage',
 'color':'color',
 'vf_Doors':'doors',
 'vf_EngineHP':'enginehp',
 'vf_FuelTypePrimary':'fueltypeprimary',
 'vf_TransmissionStyle':'transmissionstyle',
})

In [ ]:
df = df.reset_index()

In [ ]:
df.count()

In [ ]:
df = df.loc[df['askPrice'] != 0]

In [ ]:
df.count()

In [ ]:
df.head()

--------

# Data Testing

--------------

In [ ]:
# trim data into sample
df_sample = df.sample(n=10000, random_state=42)
df_sample.reset_index(inplace=True)

In [ ]:
# convert text to numerical with dummy columns
X_dummies = pd.get_dummies(
    data = df_sample, columns = [
        'bodyclass',
        'make',
        'model',
        ])

X_dummies.shape

In [ ]:
# scale the data to lessen bias
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_dummies)
X_scaled.shape

In [ ]:
# dimensionality reduction using PCA
pca = PCA(n_components=.9)
components = pca.fit_transform(X_scaled)
components.shape

In [ ]:
# dimensionality reduction using t-SNE
X_embedded = TSNE(perplexity=50).fit_transform(components)
X_embedded.shape

In [ ]:
# plot the initial Kmeans cluster diagram
fig = plt.figure(figsize = (19.20,10.80))

plt.scatter(X_embedded[:,0],X_embedded[:,1])
plt.grid()
plt.show()

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(X_embedded)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow

In [ ]:
# Plot the elbow curve to find the best candidate(s) for k

fig = plt.figure(figsize = (19.20,10.80))

plt.plot(df_elbow['k'], df_elbow['inertia'])
plt.xticks(range(1,11))
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow curve')
plt.grid()
plt.show()

In [ ]:
def get_clusters(k, data):
    # Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)

    # Train the model
    model.fit(data)

    # Predict clusters
    predictions = model.predict(data)

    # Create return DataFrame with predicted clusters
    data["class"] = model.labels_

    return data

In [ ]:
# transform embedded array into df for clustering purposes
cluster_df = pd.DataFrame(X_embedded, columns=['col_1','col_2'])

In [ ]:
# display the cluster df
cluster_df

In [ ]:
# after plotting the inertia of the K-means cluster data, 4 clusters was determined to be the best
clusters = get_clusters(4, cluster_df) 


In [ ]:
def show_clusters(df):
    fig = plt.figure(figsize = (19.20,10.80))

    plt.scatter(df['col_1'], df['col_2'], c=df['class'])
    plt.xlabel('col_1')
    plt.ylabel('col_2')
    plt.grid()
    plt.show()

In [ ]:
show_clusters(clusters)


In [ ]:
model = KMeans(n_clusters=4, random_state=0)

# Train the model
model.fit(cluster_df)

# Predict clusters
predictions = model.predict(cluster_df)

# Create return DataFrame with predicted clusters
cluster_df["class"] = model.labels_

In [ ]:
clusters

In [ ]:
df_sample['class'] = clusters['class']

In [ ]:
df_sample